Script to demo scikit for tweet popular/unpopular classification.

In [1]:
from __future__ import division
from __future__ import print_function
import csv
import datetime as dt
import os
import platform
import sys

import numpy as np
import pandas
from sklearn import preprocessing
from sklearn import svm
from sklearn import tree
from sklearn.cross_validation import train_test_split
from sklearn.externals import joblib
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report

In [2]:
def csv_to_dict_cesar(csv_filename):
    # Let's say, We are intersted in only count features
    count_features = ['_char_count', '_hashtag_count', '_word_count', '_url_count']
    with open(csv_filename) as f:
        features = [({k: int(v) for k, v in row.items() if k in count_features}, row['_popular'])
                    for row in csv.DictReader(f, skipinitialspace=True)]
        X = [f[0] for f in features]
        Y = [f[1] for f in features]
    return (X, Y)

In [3]:
def csv_to_df(csv_file):
    """Open csv with Pandas DataFrame, then convert to dict 
    and return.
    """
    
    dataframe = pandas.read_csv(csv_file, 
                                encoding='utf-8', 
                                engine='python', 
                                sep='|',
                                delimiter='|',
                                index_col=0)
    return dataframe

In [5]:
def train(csv_filename):
    
    # Open .csv and load into df
    #features = csv_to_dict_cesar(csv_filename)
    #vec = DictVectorizer()
    #data = features[0]  # list of dict: [{'_word_count': 5, '_hashtag_count': 0, '_char_count': 50, '_url_count': 0}
    #target = features[1]  # list of str: ['TRUE', 'TRUE', 'FALSE', ...]
    
    print('Loading CSV into DataFrame ...')
    t0 = dt.datetime.utcnow()
    df_features = csv_to_df(csv_filename)
    print('... finished in {} secs.'.format(dt.datetime.utcnow() - t0))
    print()
    
    # Convert df to list of dict
    # http://stackoverflow.com/a/29815523
    print('Loading DataFrame into vectorizer')
    t0 = dt.datetime.utcnow()
    print('DataFrame rows before:', len(df_features.index))
    target_1 = df_features['_popular'].tolist()  # list of str: ['TRUE', 'TRUE', 'FALSE', ...]
    #data_pivoted = df_features.T.to_dict().values()  # list of dict: [{u'_char_count': 140.0, u'_popular': u'True', ...}, ...]
    print('Boolean features before pivot:', len(target_1))
    
    # pivot table, which (for some reason) reduces rows to 18287
    data_pivoted = df_features.T.to_dict()
    data_unpivoted = df_features.T.to_dict()
    print('data_pivoted:', len(data_pivoted))
    print('data_unpivoted:', len(data_unpivoted))
    
    
    vec = DictVectorizer()
    X = vec.fit_transform(data_pivoted).toarray()  # change to numpy array
    Y = np.array(target)  # change to numpy array
    print('... finished in {} secs.'.format(dt.datetime.utcnow() - t0))
    print()

    
    '''
    -In case we need to know the features
    '''
    feature_names = vec.get_feature_names()

    '''
    -Dividing the data into train and test
    -random_state is pseudo-random number generator state used for
     random sampling
    '''
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)
    
    # write models dir if not present
    models_dir = 'models'
    if not os.path.isdir(models_dir):
        os.mkdir(models_dir)

    '''
    -PREPOCESSING 
    -Here, scaled data has zero mean and unit varience
    -We save the scaler to later use with testing/prediction data
    '''
    print('Scaling data ...')
    t0 = dt.datetime.utcnow()
    scaler = preprocessing.StandardScaler().fit(X_train)
    joblib.dump(scaler, 'models/scaler.pickle')
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    print('... finished in {} secs.'.format(dt.datetime.utcnow() - t0))
    print()
    

    '''
    -This is where we define the models
    -Here, I use SVM and Decision tree with pre-defined parameters
    -We can learn these parameters given our data
    '''
    print('Defining and fitting models ...')
    t0 = dt.datetime.utcnow()   
    clf0 = svm.LinearSVC(C=100.)
    clf1 = tree.DecisionTreeClassifier()

    clf0.fit(X_train_scaled, Y_train)
    clf1.fit(X_train_scaled, Y_train)

    joblib.dump(clf0, 'models/svc.pickle')
    joblib.dump(clf1, 'models/tree.pickle')

    print('... finished in {} secs.'.format(dt.datetime.utcnow() - t0))
    print()
    

    Y_prediction_svc = clf0.predict(X_test_scaled)
    print('svc_predictions ', Y_prediction_svc)
    Y_prediction_tree = clf1.predict(X_test_scaled)
    print('tree_predictions ', Y_prediction_tree)
    expected = Y_test
    print('actual_values   ', expected)

    '''
    Classifiation metrics
    (Case 1): SVMs
    '''
    print()
    print('----Linear SVC_report--------------------------')
    print(classification_report(expected, Y_prediction_svc))

    '''
    Classification metrics
    (case 2): Decision tree
    '''
    print()
    print('----Tree_report--------------------------------')
    print(classification_report(expected, Y_prediction_tree))

In [6]:
train("feature_tables/basics.csv")
#train("feature_tables/test.csv")

Loading CSV into DataFrame ...
... finished in 0:00:00.648989 secs.

Loading DataFrame into vectorizer
DataFrame rows before: 22709
Boolean features before pivot: 22709
data_pivoted: 18287
data_unpivoted: 18287


/Users/kyle/tensorflow/tensorflow/examples/udacity/venv27/lib/python2.7/site-packages/pandas/core/frame.py:835: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  "columns will be omitted.", UserWarning)


TypeError: float() argument must be a string or a number